## Exploratory Data Analysis - Locales en Venta en CABA

#### Importamos librerías relevantes
Pandas para manejo de data, numpy para operaciones matemáticas, matplolib y seaborn para graficar.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Locales en venta en CABA - 2020
El objetivo de este trabajo es predecir el precio de un local en la Ciudad de Buenos Aires a partir de la data disponible en el portal de información del Gobierno de la Ciudad para el 2020: https://data.buenosaires.gob.ar/dataset/locales-en-venta
Para lograr esto, en primera instancia se analizará este dataset y se lo limpiará en caso de que sea necesario.

Leemos el dataset y lo almacenamos en un dataframe:

In [10]:
df = pd.read_csv('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/secretaria-de-desarrollo-urbano/locales-en-venta/locales-en-venta-2020.csv')